In [1]:
#https://arxiv.org/abs/2011.00027
#https://github.com/KristianWold/Master-Thesis

In [2]:
import numpy as np
import matplotlib.pyplot as plt

from torchvision import datasets, transforms
from scipy.optimize import *
from logicqubit import *
from sklearn.datasets import fetch_openml
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn.preprocessing import scale, minmax_scale
from sklearn.preprocessing import MinMaxScaler
import random
from logicqubit.logic import *

Cuda is not available!
logicqubit version 1.5.8


In [3]:
iris = load_iris()

x = iris.data[0:100]
y = iris.target[0:100]

scaler = MinMaxScaler((0, pi))
x = scaler.fit_transform(x)

y = minmax_scale(y)

In [15]:
def evaluate(weights, data):
    outputs = []
    for x in data:    
        logicQuBit  = LogicQuBit(4)
        q1 = Qubit()
        q2 = Qubit()
        q3 = Qubit()
        q4 = Qubit()
        
        q1.H()
        q2.H()
        q3.H()
        q4.H()        

        # encode
        q1.RZ(x[0])
        q2.RZ(x[1])
        q3.RZ(x[2])
        q4.RZ(x[3])

        # weights
        q1.RY(weights[0])
        q2.RY(weights[1])
        q3.RY(weights[2])
        q4.RY(weights[3])

        q2.CX(q1)
        q3.CX(q2)
        q4.CX(q3)

        q1.RY(weights[4])
        q2.RY(weights[5])
        q3.RY(weights[6])
        q4.RY(weights[7])

        q2.CX(q1)
        q3.CX(q2)
        q4.CX(q3)

        # bias
        q4.RY(weights[8])

        result = logicQuBit.Measure([q4])
        value = result[1].real
        outputs.append(value)
    return np.array(outputs)

In [16]:
def gradient(weights, x, y, y_pred):
    n_samples = x.shape[0]
    n_weights = weights.shape[0]
    shift = pi/2
    cross_entropy_deriv = (y_pred - y)/(y_pred*(1-y_pred) + 1e-15)
    grad = np.zeros((n_weights, n_samples))
    
    for i in range(n_weights):
        #parameter shift rule
        shift_vect = np.array([shift if j==i else 0 for j in range(n_weights)])
        shift_right = weights + shift_vect
        shift_left = weights - shift_vect
        
        expectation_right = evaluate(shift_right, x)
        expectation_left = evaluate(shift_left, x)

        grad[i] = expectation_right - expectation_left

    gradients = 1.0/n_samples * grad.dot(cross_entropy_deriv) #inner product
    return gradients

In [17]:
weights = np.random.uniform(-np.pi, np.pi, 9)

In [18]:
lr = 1
epochs = 100

for i in range(epochs):
    y_pred = evaluate(weights, x)
    grad = gradient(weights, x, y, y_pred)
    weights = weights - lr*grad
    accuracy = np.mean(np.round(y_pred).astype(int) == y)
    print(accuracy)
    if accuracy > 0.98:
        break

0.05
0.46
0.97
1.0


In [19]:
y_pred = evaluate(weights, x)
np.round(y_pred).astype(int)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])